In [2]:
!pip install -U llama-index

  Using cached llama_index_embeddings_openai-0.1.6-py3-none-any.whl.metadata (654 bytes)
  Using cached llama_index_multi_modal_llms_openai-0.1.4-py3-none-any.whl.metadata (728 bytes)
  Using cached llama_index_program_openai-0.1.4-py3-none-any.whl.metadata (766 bytes)
  Using cached llama_index_question_gen_openai-0.1.3-py3-none-any.whl.metadata (785 bytes)
  Using cached llama_index_readers_llama_parse-0.1.3-py3-none-any.whl.metadata (3.6 kB)
  Using cached striprtf-0.0.26-py3-none-any.whl.metadata (2.1 kB)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 118.9 MB/s eta 0:00:0000:0100:01
Using cached llama_index_embeddings_openai-0.1.6-py3-none-any.whl (6.0 kB)
Using cached llama_index_multi_modal_llms_openai-0.1.4-py3-none-any.whl (5.8 kB)
Using cached llama_index_program_openai-0.1.4-py3-none-any.whl (4.1 kB)
Using cached llama_index_question_gen_openai-0.1.3-py3-none-any.whl (2.9 kB)
Using cached llama_index_readers_llama_parse-0.1.3-py3-none-any.whl (2.5 kB)
Using cached striprtf-0.0.26-py3-none-any.whl (6.9 kB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.11
    Uninstalling llama-index-core-0.10.11:
      Successfully uninstalled llama-index-core-0.10.11
  Attempting uninstall: llama-index-readers-file
    Found existing installation: llama-index-readers-file 0.1.3
    Uninstalling llama-index-readers-file-0.1.3:
      Successfully uninstalled llama-index-readers-file-0.1.3
  Attempting uninstall: llama-index-llms-openai
   

    Uninstalling llama-index-question-gen-openai-0.1.1:
      Successfully uninstalled llama-index-question-gen-openai-0.1.1
  Attempting uninstall: llama-index
    Found existing installation: llama-index 0.10.7
    Uninstalling llama-index-0.10.7:
      Successfully uninstalled llama-index-0.10.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-huggingface 0.1.1 requires torch<3.0.0,>=2.1.2, but you have torch 2.0.0 which is incompatible.
llama-index-vector-stores-chroma 0.0.1 requires llama-index-core<0.10.0,>=0.9.32, but you have llama-index-core 0.10.20.post2 which is incompatible.


In [1]:
!pip install -U llama-index-multi-modal-llms-ollama

  Using cached llama_index_multi_modal_llms_ollama-0.1.3-py3-none-any.whl.metadata (711 bytes)
  Using cached ollama-0.1.7-py3-none-any.whl.metadata (3.8 kB)
  Using cached httpx-0.25.2-py3-none-any.whl.metadata (6.9 kB)


Using cached llama_index_multi_modal_llms_ollama-0.1.3-py3-none-any.whl (3.5 kB)
Using cached ollama-0.1.7-py3-none-any.whl (9.4 kB)
Using cached httpx-0.25.2-py3-none-any.whl (74 kB)
  Attempting uninstall: httpx
    Found existing installation: httpx 0.26.0
    Uninstalling httpx-0.26.0:
      Successfully uninstalled httpx-0.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-huggingface 0.1.1 requires torch<3.0.0,>=2.1.2, but you have torch 2.0.0 which is incompatible.
llama-index-vector-stores-chroma 0.0.1 requires llama-index-core<0.10.0,>=0.9.32, but you have llama-index-core 0.10.11 which is incompatible.
